In [2]:
import os
from zipfile import ZipFile
import re
import pysrt
import numpy as np
import spacy
from tqdm import tqdm
import json
# !python -m spacy download de_core_news_sm
# !python -m spacy download ru_core_news_sm
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
data_path = './../../data/Diploma'
films = list(os.walk(data_path))[0][1]

In [7]:
films = [films[-1]]

In [9]:
film_texts = []
for film in films:
    path = os.path.join(data_path, film, 'german_annotated')
    files = list(os.walk(path))[0][2]
    for file in files:
        file_path = os.path.join(path, file)
        try:
            text = pysrt.open(file_path, encoding='utf-8')
        except UnicodeDecodeError as e:
            text = pysrt.open(file_path, encoding='latin-1')
        film_texts.append(text)

In [10]:
len(film_texts)

1

In [11]:
texts = []
for film_text in film_texts:
    text = ''
    for saying in film_text:
        if text != '':
            text += ' ' + saying.text
        else:
            text += saying.text
    texts.append(text)

In [12]:
for num, text in enumerate(texts):
    text = re.sub(r'NETFLIX PRÄSENTIERT EINE NETFLIX ORIGINAL SERIE', '', text)
    text = re.sub(r'German SDH Subtitles by\\n<i>Untertitel</i>: Gelula/SDI', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'♪.+?♪', '', text)
    text = re.sub(r'\.\.\. \.\.\.', ' ', text)
    text = re.sub(r'<i>.+?</i>', '', text)
    text = re.sub(r'\(.+?\)', '', text)
    text = re.sub(r'\[.+?\]', '', text)
    text = re.sub(r'{\\\\an8}', '', text)
    text = re.sub(r'<.+?>', '', text)
    text = re.sub(r'\*.+?\*', '', text)
    text = re.sub(r'[A-Z ]+?:', '', text)
    text = re.sub(r'\s+', ' ', text)
    texts[num] = text

In [13]:
data = []
for text in tqdm(texts):
    sents = sent_tokenize(text)
    for num, sent in enumerate(sents):
        sents[num] = re.sub('^(-)([A-Z])', '\g<1> \g<2>', sent)
    # подставляем подходящий первый контекст
    prev = [sents[0]]
    sent_num = 2
    token_num = 10
    for num, sent in enumerate(sents):
        # пропускаем первое предложение
        if num == 0:
            continue
        prev_data = []
        for prev_sent in prev[::-1]:
            prev_token_num = 0
            for prev_sent_data in prev_data:
                sent_len = len(word_tokenize(prev_sent_data))
                prev_token_num += sent_len
            prev_sent_num = len(prev_data)
            if (prev_token_num < token_num)or(prev_sent_num < sent_num):
                  prev_data.append(prev_sent)
        prev_data = prev_data[::-1]
        data.append((prev_data, sent))
        prev = prev_data + [sent]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.68it/s]


In [14]:
tokens = 0
sentences = 0
dfs = []
Bs = 0
Is = 0
Os = 0
for prev, sent in tqdm(data):
    words = word_tokenize(sent)
    for word in words:
        if word.endswith('-B'):
            Bs += 1
        elif word.endswith('-I'):
            Is += 1
        else:
            Os += 1
    tokens += len(words)
    sentences += len(sent_tokenize(sent))
    dfs_num = re.findall('[a-z]-B[^a-z]', sent)
    dfs += dfs_num

100%|████████████████████████████████████████████████████████████████████████████| 1859/1859 [00:00<00:00, 9328.49it/s]


In [15]:
print(tokens, sentences, len(dfs))

12684 1859 0


In [10]:
print(Bs, Is, Os)

959 1333 552911


In [16]:
with open('testing_model.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(data, ensure_ascii=False))